### 示例2-1

In [ ]:
import json
import pprint
f = open(r"c:\data\china\city3.json","w")
json_object = {'Beijing': {'coord': {'lat': 39.92, 'lon': 116.37}, 'pop': 2170},
            'Guangzhou': {'coord': {'lat': 23.13, 'lon': 113.25}, 'pop': 1449},
            'Shanghai': {'coord': {'lat': 31.26, 'lon': 121.53}, 'pop': 2418}}
json.dump(json_object,f)
f.close()

f = open(r"c:\data\china\city3.json","r")
value = json.load(f)
pprint.pprint(value)
f.close()

### 示例2-2

In [ ]:
from geojson import Point,Feature,FeatureCollection,dump
input_file = open(r"c:\data\china\china_cities.csv","r",encoding="utf8")
features = []
for line in input_file:
    name,lng,lat,province,isCapital = line.split(",")
    point = Point([float(lng),float(lat)])
    properties = {"城市名":name,"省名":province,"是否省会":isCapital}
    feature = Feature(geometry=point, properties=properties)
    features.append(feature)
featureCollection = FeatureCollection(features)
output_file = open(r"c:\data\china\china_cities.geojson","w")
dump(featureCollection,output_file)
input_file.close()
output_file.close()

### 示例2-3

In [ ]:
import struct
f = open(r"C:\data\USA\CAPITALS.shp","rb")
#读取文件长度
f.seek(24)
s = f.read(4)
file_length = struct.unpack(">i",s)[0]  #>表示字节顺序为big
#计算记录数
record_count = int((file_length - 50)/14)
print(f"记录数:{record_count}")
for i in range(record_count):
    f.seek(100+28*i+12)
    s = f.read(16)
    x,y = struct.unpack("dd",s)
    print(f"x:{x},y:{y}")

### 示例2-4

In [ ]:
import struct
f = open(r"C:\data\USA\STATES.shp","rb")
#读取文件长度
f.seek(24)
s = f.read(4)
file_length = struct.unpack(">i",s)[0]
#设定初始记录（第一个记录）的位置为100
record_pos = 100
while True:
    if record_pos == file_length*2:
        break
    f.seek(record_pos)
    s = f.read(8)
    record_id,content_length = struct.unpack(">ii",s)
    f.seek(record_pos+44)
    s = f.read(8)
    part_count,point_count = struct.unpack("ii",s)
    print(f"id:{record_id},part数:{part_count},点数：{point_count}")
    record_pos = record_pos + 8 + content_length*2

### 示例2-5

In [ ]:
import struct
f = open(r"c:\data\USA\STATES.shx","rb")
#读取文件长度并计算记录数
f.seek(24)
s = f.read(4)
file_length = struct.unpack(">i",s)[0]
record_count = int((file_length*2-100)/8)
record_pos = []
#读取每个记录的起始位置，并存放到一个列表中
for i in range(record_count):
    f.seek(100 + i*8)
    s = f.read(4)
    pos = struct.unpack(">i",s)[0]*2
    record_pos.append(pos)
f.close()

f = open(r"c:\data\USA\STATES.shp","rb")
for i in range(record_count):
    pos = record_pos[i]
    f.seek(pos)
    #读取记录的id
    s = f.read(8)
    record_id,content_length = struct.unpack(">ii",s)
    print(f"id:{record_id}")
    #读取part数和点数
    f.seek(pos+44)
    s = f.read(8)
    part_count,point_count = struct.unpack("ii",s)
    for i in range(part_count):
        print("part:",i+1)
        #读取第i个part起始点的索引号
        f.seek(pos + 52 + i*4)
        s = f.read(4)
        from_index = struct.unpack("i",s)[0]
        #读取第i个part结束点的索引号，
        #如果是最后一个part，结束点的索引号是点数
        if i == (part_count - 1):
            end_index = point_count
        else:
            f.seek(pos + 52 + (i+1)*4)
            s = f.read(4)
            end_index = struct.unpack("i",s)[0]
        #读取当前part中每个点的x、y坐标       
        for j in range(end_index-from_index):
            x_pos = pos + 52 + part_count*4 + (from_index + j)*16
            f.seek(x_pos)
            s = f.read(16)
            x,y = struct.unpack("dd",s)
            print(f"x:{x},y:{y}")
f.close()

### 示例2-6

In [ ]:
import struct
f = open(r"C:\data\USA\USHIGH.dbf","rb")
#读取记录数
f.seek(4)
s = f.read(4)
record_count = struct.unpack("i",s)[0]
#读取第一个记录的位置（即文件头和字段描述的长度），
#并计算字段数（单个字段的长度为32）
f.seek(8)
s = f.read(2)
record_pos = struct.unpack("H",s)[0]
#33是文件头长度（32）加上字段描述信息中的结束标记字节
field_count = int((record_pos - 33)/32)
#读取一个记录的长度（所有记录的长度一致）
f.seek(10)
s = f.read(2)
record_length = struct.unpack("H",s)[0]
field_name_list = []
field_length_list = []
#读取每个字段的名称和长度,并追加到列表中
for i in range(field_count):
    pos = 32+i*32
    f.seek(pos)
    s = f.read(10)
    field_name = struct.unpack("10c",s)
    field_name = b"".join(field_name).strip(b"\x00")
    field_name_list.append(field_name.decode())
    f.seek(pos+16)
    s = f.read(1)
    field_length = struct.unpack("B",s)[0]
    field_length_list.append(field_length)

#读取每个记录的字段值    
for i in range(record_count):
    print(f"record {i+1}")
    f.seek(record_pos+i*record_length+1)
    s = f.read(record_length-1)
    field_value = struct.unpack(f"{record_length-1}c",s)
    #把元组中的元素（字节字符）连接成字节字符串
    d = b"".join(field_value)
    #根据每个字段的长度从字节字符串中切片出字段值
    start = 0
    for k in range(field_count):
        end = start + field_length_list[k]
        field_value = d[start:end].strip().decode()
        field_name = field_name_list[k]
        print(f"{field_name}:{field_value}")
        start = start + field_length_list[k]

### 示例2-7

In [ ]:
import struct
import cv2
import numpy as np
#创建一个300行*300列，波段数为3，像元值位数为8的tif图像数据
img = np.random.randint(0,255,[5,5,3],dtype=np.uint8)
cv2.imwrite(r"c:\data\tmp\test.tif",img)

f = open(r"c:\data\tmp\test.tif","rb")
#读取第一个IFD的位置
f.seek(4)
s = f.read(4)
IFD_pos = struct.unpack("i",s)[0]
#读取标签数
f.seek(IFD_pos)
s = f.read(2)
tag_count = struct.unpack("H",s)[0]
print(f"标签数:{tag_count}")
#循环读每个标签
for i in range(tag_count):
    #读取标签ID
    f.seek(IFD_pos+2+i*12)
    s = f.read(2)
    tag_id = struct.unpack("H",s)[0]
    #读取标签值类型
    f.seek(IFD_pos+4+i*12)
    s = f.read(2)
    value_type = struct.unpack("H",s)[0]
    #读取标签值个数
    f.seek(IFD_pos+6+i*12)
    s = f.read(4)
    value_count = struct.unpack("L",s)[0]
    #定位标签值位置
    f.seek(IFD_pos+10+i*12)
    #如id为256，则标签值记录列数
    if tag_id == 256:
        s = f.read(4)
        columns = struct.unpack("i",s)[0]
        print(f"列数:{columns}")
    #如id为257，则标签值记录行数
    elif tag_id == 257:
        s = f.read(4)
        rows = struct.unpack("i",s)[0]
        print(f"行数:{rows}")
    #如id为258，则标签值记录每个分量（波段）像元值的位数或记录位数的地址
    elif tag_id == 258:
        #判断值的个数乘上每个值的字节数是否小于等于4
        if value_count*2 <= 4:
            s = f.read(value_count*2)
            bits = struct.unpack(f"{value_count}H",s)
        else:
            s = f.read(4)
            pos = struct.unpack("i",s)[0]
            f.seek(pos)
            s = f.read(value_count*2)
            bits = struct.unpack(f"{value_count}H",s)
        print(f"位数:{bits}")
    #如id为277，则标签值记录波段数
    elif tag_id == 277:
        s = f.read(4)
        band_count = struct.unpack("i",s)[0]
        print(f"波段数:{band_count}")
    else:
        continue

### 示例2-8

In [ ]:
import struct
f = open(r"c:\data\landsat\sub_image.tif","rb")
#读取第一个IFD的位置
f.seek(4)
s = f.read(4)
IFD_pos = struct.unpack("i",s)[0]
#读取标签数
f.seek(IFD_pos)
s = f.read(2)
tag_count = struct.unpack("H",s)[0]
#读取每个标签
tag_ids = []
for i in range(tag_count):
    f.seek(IFD_pos+2+i*12)
    s = f.read(2)
    tag_id = struct.unpack("H",s)[0]
    tag_ids.append(tag_id)
    f.seek(IFD_pos+6+i*12)
    s = f.read(4)
    value_count = struct.unpack("L",s)[0]
    f.seek(IFD_pos+10+i*12)
    s = f.read(4)
    value_pos = struct.unpack("I",s)[0]
    f.seek(value_pos)
    #读X、Y、Z方向比例尺
    if tag_id==33550:
        s = f.read(24)
        value = struct.unpack("3d",s)
        print(f"X、Y、Z方向比例尺：")
        print(value)
    #读控制点的栅格空间坐标和地理空间坐标
    if tag_id==33922:
        s = f.read(value_count*8)
        value = struct.unpack(f"{value_count}d",s)
        print("控制点的栅格空间坐标和地理空间坐标：")
        print(value)
    #读转换方程系数
    if tag_id==34264:
        s = f.read(value_count*8)
        value = struct.unpack(f"{value_count}d",s)
        print("转换方程系数：")
        print(value)
    #读元标签
    if tag_id==34735:
        s = f.read(value_count*2)
        value = struct.unpack(f"{value_count}H",s)
        print("元标签：")
        print(value)
    #读取GeoDoubleParamsTag中的值
    if tag_id==34736:
        s = f.read(value_count*8)
        value = struct.unpack(f"{value_count}d",s)
        print("数字型标签值：")
        print(value)
    #读取GeoAsciiParamsTag中的值
    if tag_id==34737:
        s = f.read(value_count)
        value = struct.unpack(f"{value_count}s",s)
        print("字符串型标签值：")
        print(value)